In [2]:
##Import packages

import subprocess
import numpy as np
import os, sys, shutil
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import seaborn as sns

import pandas as pd
import gc
import math
import time

from sklearn.cluster import KMeans

%matplotlib inline

In [3]:
def gaussian(x,mu,sig):
    return np.exp(-((x-mu)**2)/(2*sig**2))

In [4]:
# 0. Import## Identify all water molecules
def distance(box, atom, aim_pool):
    vecs = abs(np.array(aim_pool, dtype='float')-np.array(atom, dtype='float'))
    for i in range(3):
        vecs[:,i] = np.minimum(vecs[:,i],box[i]-vecs[:,i])
    distance = np.linalg.norm(vecs,axis=1)
    return distance
def findNearest(box, atom, aim_pool, num):
    """
    Find the nearest one to given atom from a group of candidate
    Input: 
        box: periodic box size
        atom: given atom (position, dimention=3d)
        aim pool: a list of positions.
    Return:
        index of closest atom in aim_pool
    
    """
    vecs = abs(np.array(aim_pool, dtype='float')-np.array(atom, dtype='float'))
    for i in range(3):
        vecs[:,i] = np.minimum(vecs[:,i],box[i]-vecs[:,i])
    distance = np.linalg.norm(vecs,axis=1)
    sorted_dis = sorted(distance)
    res = np.where(distance <= sorted_dis[num])[0]
    if len(res)>num:
        res = np.concatenate((np.where(distance < sorted_dis[num])[0],np.where(distance == sorted_dis[num])[0]))
        res = res[:num]
    return res

In [5]:
def get_O_in_OH(atoms_lines, box, eles):
    
    pos = [[float(_e) for _e in e.split()] for e in atoms_lines[1:]]
    pos = np.array(pos, dtype = 'object')
    atoms = np.concatenate((eles, pos),axis=1)
    for i in range(len(box)):
        atoms[:,i+1] = atoms[:,i+1]*box[i]
    
    Os = atoms[np.where(atoms[:,0]=='O')[0],1:]
    Hs = atoms[np.where(atoms[:,0]=='H')[0],1:]

    dis_mat = np.zeros((len(Hs), len(Os)))
    for i in range(len(Hs)):
        dis_mat[i] = distance(box, Hs[i], Os)
    dis = np.array([np.where(e == min(e))[0][0] for e in dis_mat])
    return dis

In [6]:
def _if_bond_(lines, Os, Hs, box):
    
    OH_cutoff = 1.28
    
    pos = [[float(_e) for _e in e.split()] for e in lines[1:]]
    pos = np.array(pos, dtype = 'object')
    for i in range(len(box)):
        pos[:,i] = pos[:,i]*box[i]
    
    i = 0
    l = []
    ang = []
    
    
    def __periodic(vec,box):
        for i,e in enumerate(vec):
            if abs(e)>box[i]/2:
                vec[i] = vec[i]-vec[i]/abs(vec[i])*box[i]
        return vec

    def __angle(vec1, vec2):
        a = vec1/np.linalg.norm(vec1)
        b = vec2/np.linalg.norm(vec2)
        temp = np.dot(vec1, vec2)
        if temp>1:
            temp=1
        if temp<-1:
            temp=-1
        return math.acos(temp)
    
    while i < len(Os):
        _O = Os[i]
        _num = np.count_nonzero(Os == _O, axis=0)
        vec = []
        for j in range(_num):
            _i = i+j 
            _temp = np.linalg.norm(__periodic(pos[Hs[_i]]-pos[Os[_i]],box))
            if _temp <= OH_cutoff:
                l.append(1)
                vec.append(1)
            else:
                l.append(0)
                vec.append(0)
        if _num > 1:
            temp = np.zeros((_num,_num))
            for _i in range(_num):
                for _j in range(_i+1,_num):
                    ang.append(vec[_i]*vec[_j])
        i=i+_num
    return np.array(l), np.array(ang)
                    
        

In [7]:
def bond_angle(lines, Os, Hs, box):
    
    pos = [[float(_e) for _e in e.split()] for e in lines[1:]]
    pos = np.array(pos, dtype = 'object')
    for i in range(len(box)):
        pos[:,i] = pos[:,i]*box[i]
    
    i = 0
    l = []
    ang = []
    
    
    def __periodic(vec,box):
        for i,e in enumerate(vec):
            if abs(e)>box[i]/2:
                vec[i] = vec[i]-vec[i]/abs(vec[i])*box[i]
        return vec

    def __angle(vec1, vec2):
        a = vec1/np.linalg.norm(vec1)
        b = vec2/np.linalg.norm(vec2)
        temp = np.dot(vec1, vec2)
        if temp>1:
            temp=1
        if temp<-1:
            temp=-1
        return math.acos(temp)
    
    while i < len(Os):
        _O = Os[i]
        _num = np.count_nonzero(Os == _O, axis=0)
        vec = []
        for j in range(_num):
            _i = i+j
            vec.append(__periodic(pos[Hs[_i]]-pos[Os[_i]],box)) 
            l.append(np.linalg.norm(vec[-1]))
        if _num > 1:
            temp = np.zeros((_num,_num))
            for _i in range(_num):
                for _j in range(_i+1,_num):
                    ang.append(__angle(vec[_i],vec[_j]))
        i=i+_num
    return np.array(l), np.array(ang)

In [15]:
def water_form(lines, eles, box):  # , res_type = 'idxs'):
    #mol_type_class = {'-OH in H3PO4':6, '-O in H3PO4':7, 'hydronium-AAA':4, 
    #                    'water-DDA':2, 'water-DA':0, 'water-DDAA':3, 'water-DAA':1}
    mol_type_class = {'-OH in H3PO4':6, '-O in H3PO4':7, 'hydronium-AAA':4, 
                        'water-DDA':2, 'water-DA':0, 'water-DDAA':3, 'water-DAA':1}
    
    pos = [[float(_e) for _e in e.split()] for e in lines[1:]]
    pos = np.array(pos, dtype = 'object')
    atoms = np.concatenate((eles, pos),axis=1)
    for i in range(len(box)):
        atoms[:,i+1] = atoms[:,i+1]*box[i]
        
    Oinds = np.where(atoms[:,0]=='O')[0]
    Hinds = np.where(atoms[:,0]=='H')[0]
    Linds = np.where(atoms[:,0]=='Li')[0]
    Cinds = np.where(atoms[:,0]=='Cl')[0]

    Os = atoms[Oinds,1:]
    Hs = atoms[Hinds,1:]
    Ls = atoms[Linds,1:]
    Cs = atoms[Cinds,1:]

    
    '''
    pair_coeff 1 1 0.07647 2.126 # Li
    pair_coeff 2 2 0.4928 4.417 # Cl
    pair_coeff 3 3 0.1553 3.166      # O
    pair_coeff 4 4 0.0460    0.4000      # H
    pair_coeff 3 4 0.0    0.0 #OH
    
    '''
    OH_cutoff = 1.28
    O_H_cutoff = 2.25     # O...H < 2.25 
    O_Li_cutoff = 2.33  
    H_Cl_cutoff = 2.441  
    OO_cutoff = 3.4       # O-O < 3.4 A   
    
    # H_Cl
    # https://pubs.acs.org/doi/10.1021/jp310596c
    # H-Cl: 2.441        O-H...Cl 119.0
        
    mol = []
    OO_mat = np.zeros(( len(Os) , len(Os) ))
    HinWater = np.zeros(( len(Os) , 2 , 3 ))
    


    for i in range(len(Os)):
        temp = distance(box, Os[i], Os)
        for _i in np.where(temp<=OO_cutoff)[0]:
            if i == _i:
                continue
            OO_mat[i,_i] = 1
        HinWater[i] = Hs[findNearest(box, Os[i], Hs, 2)]
    for i in range(len(Os)):
        
        ## A
        pot_Os = Os[np.where(OO_mat[i]==1)[0]]
        
        pot_Hs = np.concatenate(HinWater[np.where(OO_mat[i]==1)[0]])
        temp_OH = distance(box, Os[i], pot_Hs)
        No_O_H = len(np.where(temp_OH<O_H_cutoff)[0])
        
        t = []
        t.append(No_O_H)
        
        
        ## D
        doner_H = 0
        #print(i)
        for _H in HinWater[i]:
            temp_HO = distance(box, _H, pot_Os)
            #print(sorted(temp_HO))
            temp_HO = np.where(temp_HO<O_H_cutoff)[0]
            doner_H += (len(temp_HO))
        t.append(doner_H)
        
        ## A-Li
        temp_OL = distance(box, Os[i], Ls)
        OL_idx = np.where(temp_OL<O_Li_cutoff)[0]
        No_OL = len(OL_idx)
        t.append(No_OL)
        
        
        ## D-Cl
        doner_H = 0
        #print(i)
        for _H in HinWater[i]:
            temp_HC = distance(box, _H, Cs)
            #print(sorted(temp_HO))
            temp_HC = np.where(temp_HC<H_Cl_cutoff)[0]
            doner_H += (len(temp_HC)-1)
        #print(doner_H)
        t.append(doner_H)
        
        mol.append(t)
        
    mol = np.array(mol,dtype='object')
    
    mol_type = ['']*len(Os)
    for i, wat in enumerate(mol):
        temp = ''
        temp = temp+'C'*wat[3]
        temp = temp+'D'*wat[1]
        temp = temp+'A'*wat[0]
        temp = temp+'L'*wat[2]
        mol_type[i] = temp
        '''
        if res_type == 'idxs':
            if temp not in mol_type_class:
                mol_type[i] = 5
            else:
                mol_type[i] = mol_type_class[temp]
        else:
            mol_type[i] = temp
        '''
    return mol_type

In [41]:
def water_form_lifetime(mol_type_time, water_form_idx = 1, water_form = 'water-DDAA'):
    
    mol_type_class = {'-OH in H3PO4':6, '-O in H3PO4':7, 'hydronium-AAA':4, 
                        'water-DDA':2, 'water-DA':0, 'water-DDAA':3, 'water-DAA':1}
    if water_form_idx == -1:
        form_idx = mol_type_class[water_form]
    else:
        form_idx = water_form
    periods = []
    periods_statis = []
    for i in range(len(mol_type_time[0])):
        l = mol_type_time[:,i]
        _start=0
        _end = 0
        _if = 0
        _periods = []
        for _i, e in enumerate(l):
            if e == form_idx:
                if _if == 0:
                    _start = _i
                    _if=1
                if _if == 1:
                    _end = _i
            if e != form_idx:
                if _if == 1:
                    _period = _end - _start
                    _if=0
                    if _period > 50:
                        _periods.append([_start,_end])
                        periods_statis.append(_end - _start)
            #print(_start,_end,_if,e)
        if _if == 1:
            _period = _end - _start
            if _period > 50:
                _periods.append([_start,_end])
                periods_statis.append(_end - _start)
        #print(i,_periods)
        periods.append(_periods)
    return periods, periods_statis

In [20]:
def generate_data(Tem = 'RT',sys_ind = '0',temp_tag='_01'):

    data_path = '/rhome/wzhan097/bigdata/proton_battery/LiCl/25p_mass/'+Tem+'/'+sys_ind+'case/dft/_history'
    #filename = 'XDATCAR'
    filename = 'XDATCAR01'
    
    
    ###
    #Reading File
    
    ###

    with open(os.path.join(data_path,filename),'r') as f:
        lines = f.readlines()
    box = lines[2:5]
    box =[e.split() for e in box]
    box = [box[0][0],box[1][1],box[2][2]]
    box = [float(e) for e in box]
    lines = lines[5:]
    types = lines[0].split()
    temp = lines[1].split()
    temp = [int(e) for e in temp]
    eles = []
    atom_num = sum(temp)

    for _i,_j in zip(types, temp):
        eles = eles+[[_i]]*_j
    eles = np.array(eles, dtype = 'object')
    lines = lines[2:]
    
    
    ###
    #Generating formation info
    ###
    
    
    O_H_mat = np.zeros((atom_num, atom_num))

    pos = [[float(_e) for _e in e.split()] for e in lines[1:1+atom_num]]
    pos = np.array(pos, dtype = 'object')
    atoms = np.concatenate((eles, pos),axis=1)

    for i in range(len(box)):
        atoms[:,i+1] = atoms[:,i+1]*box[i]

    Oinds = np.where(atoms[:,0]=='O')[0]
    Hinds = np.where(atoms[:,0]=='H')[0]
    
    
    print('generating formation info....')
    
    for i in range( 0,int(len(lines)/(atom_num+1)) ):
        temp = get_O_in_OH(lines[i*(atom_num+1):(i+1)*(atom_num+1)],box,eles)
        for _i, _j in enumerate(temp):
            O_H_mat[Oinds[_j], Hinds[_i]]+=1
        if i%500 == 0:
            print(i)

            
    ###
    #Generating bond info
    ###
    
    
    bond_Os, bond_Hs = np.where(O_H_mat!=0)
    bond_l = len(bond_Os)
    bond_a = 0
    for _O in set(bond_Os):
        bond_a += sum(list(range(np.count_nonzero(bond_Os == _O, axis=0))))
    bond_l = np.zeros(((int(len(lines)/(atom_num+1))), bond_l))
    bond_a = np.zeros(((int(len(lines)/(atom_num+1))), bond_a))
    
    print('generating bond/angle info....')
    
    for i in range( 0,int(len(lines)/(atom_num+1)) ):
        _x,_y = bond_angle(lines[i*(atom_num+1):(i+1)*(atom_num+1)], bond_Os, bond_Hs, box)
        bond_l[i] = _x
        bond_a[i] = _y
        if i%500 == 0:
            print(i)
    outfile = 'LiCl/'+sys_ind+'/'+Tem+"_OH"+temp_tag+".npy"
    np.save(outfile,bond_l)
    outfile = 'LiCl/'+sys_ind+'/'+Tem+"_HOH"+temp_tag+".npy"
    np.save(outfile,bond_a)
    
    bond_Os, bond_Hs = np.where(O_H_mat!=0)
    bond_l = len(bond_Os)
    bond_a = 0
    for _O in set(bond_Os):
        bond_a += sum(list(range(np.count_nonzero(bond_Os == _O, axis=0))))
    bond_l = np.zeros(((int(len(lines)/(atom_num+1))), bond_l))
    bond_a = np.zeros(((int(len(lines)/(atom_num+1))), bond_a))
    for i in range( 0,int(len(lines)/(atom_num+1)) ):
        _x,_y = _if_bond_(lines[i*(atom_num+1):(i+1)*(atom_num+1)], bond_Os, bond_Hs, box)
        bond_l[i] = _x
        bond_a[i] = _y
        if i%500 == 0:
            print(i)
    outfile = 'LiCl/'+sys_ind+'/'+Tem+"_OH_flg"+temp_tag+".npy"
    np.save(outfile,bond_l)
    outfile = 'LiCl/'+sys_ind+'/'+Tem+"_HOH_flg"+temp_tag+".npy"
    np.save(outfile,bond_a)
    
    ###
    #Generating lifetime info
    ###   
    
    print('generating formation lifetime info....')
    
    mol_type_time = np.zeros((int(len(lines)/(atom_num+1)), len(np.where(eles=='O')[0])))
    for i in range( 0,int(len(lines)/(atom_num+1)) ):
        temp = water_form(lines[i*(atom_num+1):(i+1)*(atom_num+1)], eles, box)
        mol_type_time[i] = temp
        if i%500 == 0:
            print(i)
    outfile = 'LiCl/'+sys_ind+'/'+Tem+"_O_form"+temp_tag+".npy"
    np.save(outfile,mol_type_time)
    outfile = 'LiCl/'+sys_ind+'/'+Tem+"_O_inds"+temp_tag+".npy"
    np.save(outfile,bond_Os)
    
    

In [54]:
def temp_get_data(sys_ind = '1',Tem = 'RT',temp_tag='_02',water_form = 'water-DDAA',w_length_plot = np.linspace(2500,4500,300)):
    
    light_c = 29979245800
    
    mol_type_time = np.load('LiCl/'+sys_ind+'/'+Tem+"_O_form"+temp_tag+".npy", allow_pickle=True)

    period,_ = water_form_lifetime(mol_type_time, water_form = water_form)
    Os = if_OH_bond = np.load('LiCl/'+sys_ind+'/'+Tem+"_O_inds"+'_para_01'+".npy")


    angles_tag = []
    for i in set(Os):
        n = len(np.where(Os==i)[0])
        _n = 0
        for _i in range(n):
            _n+=_i
        angles_tag +=[i]*_n
    angles_tag = np.array(angles_tag)

    outfile = 'LiCl/'+sys_ind+'/'+Tem+"_OH_flg"+'_para_01'+".npy"
    if_OH_bond = np.load(outfile)
    outfile = 'LiCl/'+sys_ind+'/'+Tem+"_HOH_flg"+'_para_01'+".npy"
    if_HOH_angle = np.load(outfile)
    outfile = 'LiCl/'+sys_ind+'/'+Tem+"_OH"+'_para_01'+".npy"
    OH_bond = np.load(outfile)
    outfile = 'LiCl/'+sys_ind+'/'+Tem+"_HOH"+'_para_01'+".npy"
    HOH_angle = np.load(outfile)
    bond_mat = []
    angle_mat = []
    bond_list = []
    angle_list = []
    for i,ps in enumerate(period):
        #print(i)
        for p in ps:
            _flg = if_OH_bond[p[0]:p[1],np.where(Os==i)[0]]
            _bondL =  OH_bond[p[0]:p[1],np.where(Os==i)[0]]
            _i = np.where(_flg[0]==1)[0]

            for l in _bondL[:,_i].T:
                bond_list.append([i,l])

            _flg = if_HOH_angle[p[0]:p[1],np.where(angles_tag==i)[0]]
            _angleA =  HOH_angle[p[0]:p[1],np.where(angles_tag==i)[0]]
            _i = np.where(_flg[0]==1)[0]
            for l in _angleA[:,_i].T:
                angle_list.append([i,l])
                
    x_int = np.linspace(2500, 4500, 20000)
    y_int = np.zeros(x_int.shape,dtype='float64')

    bond_list = np.array(bond_list)
    res_data = np.zeros(w_length_plot.shape)
    for idx in range(len(bond_list)):#range(len(period_len)):
        temp_data=bond_list[idx,1]  
        vel = temp_data[1:]-temp_data[:-1]
        fft = np.fft.fft(vel)
        ps = fft*np.conjugate(fft)
        acf = np.fft.ifft(ps)

        signal = vel #divide by maximum abs(vel)
        fourier = np.fft.fft(signal)
        n = signal.size
        timestep = 1E-3
        freq = 2*np.fft.fftfreq(n, d=timestep)#*np.sqrt(17/9)

        w_length = (freq*1e12)/light_c #
        for dp_x,dp_y in zip(w_length,ps):
            res_data+=dp_y.real*gaussian(w_length_plot,dp_x,10*dp_y.real)#/len(bond_list[idx,-1])
    
    angle_list = np.array(angle_list)
    res_data_ang = np.zeros(w_length_plot.shape)
    
    for idx in range(len(angle_list)):#range(len(period_len)):
        temp_data=angle_list[idx,1]  
        vel = temp_data[1:]-temp_data[:-1]
        fft = np.fft.fft(vel)
        ps = fft*np.conjugate(fft)
        acf = np.fft.ifft(ps)

        signal = vel #divide by maximum abs(vel)
        fourier = np.fft.fft(signal)
        n = signal.size
        timestep = 1E-3
        freq = 2*np.fft.fftfreq(n, d=timestep)#*np.sqrt(17/9)

        w_length = (freq*1e12)/light_c #
        for dp_x,dp_y in zip(w_length,ps):
            res_data_ang+=dp_y.real*gaussian(w_length_plot,dp_x,10*dp_y.real)#/len(angle_list[idx,-1])
            
    bond_list = np.array(bond_list)
    if not bond_list:
        return res_data, res_data_ang
    period_len = np.array([len(e) for e in bond_list[:,-1]])
    #plt.plot(w_length_plot,res_data,label='plot/DDAA/'+'RT_02')
    #plt.xlim(2500,4500)
        
    return res_data/sum(period_len), res_data_ang/sum(np.array([len(e) for e in bond_list[:,-1]]))
    return res_data, res_data_ang






In [22]:
def generate_data_chg_para(Tem = 'RT',sys_ind = '0',temp_tag='_01'):

    data_path = '/rhome/wzhan097/bigdata/proton_battery/LiCl/25p_mass/'+Tem+'/'+sys_ind+'case/dft/_history'
    #filename = 'XDATCAR'
    filename = 'XDATCAR01'
    
    
    ###
    #Reading File
    
    ###

    with open(os.path.join(data_path,filename),'r') as f:
        lines = f.readlines()
    box = lines[2:5]
    box =[e.split() for e in box]
    box = [box[0][0],box[1][1],box[2][2]]
    box = [float(e) for e in box]
    lines = lines[5:]
    types = lines[0].split()
    temp = lines[1].split()
    temp = [int(e) for e in temp]
    eles = []
    atom_num = sum(temp)

    for _i,_j in zip(types, temp):
        eles = eles+[[_i]]*_j
    eles = np.array(eles, dtype = 'object')
    lines = lines[2:]
    
    print('generating formation lifetime info....')
    
    mol_type_time = np.zeros((int(len(lines)/(atom_num+1)), len(np.where(eles=='O')[0])))
    mol_type_time = np.array(mol_type_time,dtype = 'object')
    for i in range( 0,int(len(lines)/(atom_num+1)) ):
        temp = water_form(lines[i*(atom_num+1):(i+1)*(atom_num+1)], eles, box)
        mol_type_time[i] = temp
        if i%500 == 0:
            print(i)
    outfile = 'LiCl/'+sys_ind+'/'+Tem+"_O_form"+temp_tag+".npy"
    np.save(outfile,mol_type_time)
    #outfile = 'LiCl/'+sys_ind+'/'+Tem+"_O_inds"+temp_tag+".npy"
    #np.save(outfile,bond_Os)
    
    

In [ ]:
for i in [2]: #range(1,7):
    for e in ['LT','RT']:
        generate_data(Tem = e,sys_ind = str(i),temp_tag='_para_01')

In [56]:
for i in [2]: #range(1,7):
    for e in ['RT']:
        generate_data_chg_para(Tem = e,sys_ind = str(i),temp_tag='_para_01')

generating formation lifetime info....
0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000


NameError: name 'bond_Os' is not defined

In [ ]:
from collections import Counter

temp_tag='_para_01'

res = [[],[],[],[],[],[]]
res_ang = [[],[],[],[],[],[]]
w_length_plot = np.linspace(1200,5000,1000)
colors = ['r','y','g','b','c']
marker = ['-.','--']
fig = plt.figure(figsize=(8,6))
for sys_id in range(2,3):
    for t, e in enumerate(['LT','RT']):
        x = np.load('LiCl/'+str(sys_id)+'/'+e+'_O_form_para_01.npy', allow_pickle=True)
        data_all = np.zeros(w_length_plot.shape)
        for c, water in enumerate(['DDD']):#Counter(x.flatten()).keys()):
            print('0'+str(sys_id)+'/'+e+'/'+water)
            
            x, ang_x = temp_get_data(sys_ind = str(sys_id),Tem = e ,temp_tag='_para_01',water_form = water,w_length_plot = w_length_plot)
            plt.plot(w_length_plot,x,label= e+'/'+water, color=colors[c%5],linestyle=marker[t])
            temp = np.zeros((3,len(w_length_plot)))
            temp[0] = np.array(w_length_plot)
            temp[1] = np.array(x)
            temp[2] = np.array(ang_x)
            data_all+=x
            np.save('LiCl-vib'+'/'+str(sys_id)+'/'+e+'/'+water+'_x-bond-ang'+temp_tag+'.npy',temp)
            '''
            data = np.load('LiCl-vib'+'/'+str(sys_id)+'/'+e+'/'+water+'_x-bond-ang'+temp_tag+'.npy')
            plt.plot(data[0],data[1],label='0'+str(sys_id)+'/'+e+'/'+water, color=colors[c],linestyle=marker[t])
            res[sys_id].append(x)
            res_ang[sys_id].append(ang_x)
            '''
        plt.plot(w_length_plot,data_all,label = e)
    plt.legend()
    plt.xlim(2200,4000)
    plt.show()

In [51]:
x = np.load('LiCl/2/RT_O_form_para_01.npy', allow_pickle=True)


In [36]:
from collections import Counter
Counter(x.flatten()).keys()

dict_keys(['DDAL', 'DDAA', 'DL', 'DAA', 'DDL', 'DAL', 'L', 'DA', 'AA', 'D', 'DDA', 'AL', 'DDAAA', '', 'DAAA', 'DD', 'DDAAL', 'A', 'DDDAA', 'AAA', 'DDD', 'DAAL', 'DDDAL'])

In [50]:
Counter(x.flatten())

Counter({'DDAL': 78435,
         'DDAA': 183918,
         'DL': 53870,
         'DAA': 158479,
         'DDL': 35287,
         'DAL': 77480,
         'L': 6645,
         'DA': 26852,
         'AA': 10723,
         'D': 1084,
         'DDA': 9173,
         'AL': 8172,
         'DDAAA': 8235,
         '': 140,
         'DAAA': 5135,
         'DD': 402,
         'DDAAL': 3317,
         'A': 5968,
         'DDDAA': 145,
         'AAA': 482,
         'DDD': 30,
         'DAAL': 189,
         'DDDAL': 15})

In [52]:
Counter(x.flatten())

Counter({'DDA': 37478,
         'DAAA': 5797,
         'AA': 28867,
         'DL': 50292,
         'DDAA': 122614,
         'DDAL': 71135,
         'DAL': 69717,
         'DAA': 127141,
         'AL': 18027,
         'DDL': 33742,
         'D': 2215,
         'DA': 59501,
         'A': 17938,
         'DDAAA': 7440,
         'AAA': 301,
         '': 913,
         'L': 9035,
         'DDDL': 154,
         'DD': 888,
         'DDDAA': 1017,
         'DAAL': 1787,
         'DDDAL': 309,
         'DDDA': 187,
         'DDDAAA': 128,
         'DDAAL': 1208,
         'DLL': 2127,
         'AAL': 210,
         'DDDDAL': 8,
         'DALL': 25,
         'DDLL': 3928,
         'DDDLL': 32,
         'DDDDAA': 15})